# Introdução

In [ ]:
import os
import time
import random
import shutil
import requests

import pandas as pd
import geopandas as gpd

from osgeo import ogr
from zipfile import ZipFile
from bs4 import BeautifulSoup

<br>

## Funções

In [ ]:
def get_download(url, filename, path):
    # Define o nome do arquivo, com diretório, que será salvo
    zipfile = os.path.join(path, filename)

    # Faz o download do arquivo da 'url' e salva localmente com o nome do arquivo
    r = requests.get(url, stream=True)
    with open(zipfile, 'wb') as out:
        for chunk in r.iter_content(chunk_size=128):
            out.write(chunk)
    print('File "{}" download in "{}" directory.'.format(filename, path))

In [ ]:
def unzip(filename, path):
    # Get dir of zipfile
    zipfile = os.path.join(path, filename)
    path = os.path.dirname(zipfile)

    # Create a ZipFile Object
    with ZipFile(zipfile, 'r') as zip_obj:
        # Get a list of all archived file names from the zip
        list_files = []
        list_files = zip_obj.namelist()

        # Iterate over the file names
        for file in list_files:
            name, ext = os.path.splitext(file)
            zip_obj.extract(file, os.path.dirname(zipfile))
            os.rename(os.path.join(path, file), os.path.join(path, os.path.splitext(filename)[0]+ext))
            print('File "{}" extracted as "{}" in "{}" directory.'.format(file, os.path.splitext(filename)[0]+ext, path))

    # Remove file
    os.remove(zipfile)

In [ ]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        '/': '-',
        ' Da ': ' da ',
        ' Do ': ' do ',
        ' Das ': ' das ',
        ' Dos ': ' dos ',
        '4ªcpamb': '4ª Cpamb',
        'Pelpamb': 'Pelotão Ambiental',
        'Cpamb': 'Companhia Ambiental',
        'Pres.Prudente': 'Presidente Prudente',
        'Pres. Prudente': 'Presidente Prudente',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

<br>

# Dados Espaciais: DataGeo

## *Download*

In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/CTR_NRPP/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=CTR_NRPP'
filename = 'sp_cfb.zip'
path = os.path.join('data', 'shps')

# Get Data
get_download(url, filename, path)
unzip(filename, path)

In [ ]:
# Lê arquivo
file = 'sp_cfb'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Renomeia colunas
gdf.rename(
    columns={
        'Codigo': 'id_ctr',
        'Nome'  : 'ctr_nome',
    },
    inplace=True
)

# Renomeia Campos Coluna
#gdf.loc[:, 'repdec_cod'] = gdf['repdec_cod'].astype(str).apply(lambda x: rename_nome(x))
#gdf.loc[:, 'repdec_regiao'] = gdf['repdec_regiao'].astype(str).apply(lambda x: rename_nome(x))

# Results
display(gdf.head())
gdf.plot()

# Cria Cópia
gdf_cpamb = gdf

In [ ]:
# Deleta Shapefile
file = 'sp_cfb'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
driver = ogr.GetDriverByName("ESRI Shapefile")

# Delete shapefile
if os.path.exists(shp):
    driver.DeleteDataSource(shp)

# Delete other files
for i in os.listdir(os.path.dirname(shp)):
    if i.split('.')[0] == file:
        os.remove(os.path.join('data', 'shps', i))

<br>

## Convert to Points

In [ ]:
# Lê arquivo
gdf = gpd.read_file(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/shps/sp_050k_wgs84.geojson',
)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Pega centroid
gdf_geometry = gdf.representative_point()
gdf['geometry'] = gdf_geometry

# Results
display(gdf.head())
gdf.plot()

<br>

## Spatial Join

In [ ]:
# Intersect
gdf_inter = gpd.sjoin(gdf, gdf_cpamb, how='inner', op='intersects')

# Convert to Dataframe
df = gdf_inter[['id_municipio', 'id_ctr', 'ctr_nome']].copy()

# Results
display(df.head())
gdf_inter.plot()

<br>

## Join Municípios

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Resultados
df.head()

<br>

## Join CTR

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_defesa = pd.read_csv(
    os.path.join('data', 'tabs', 'tab_ctr.csv'),
)

# Merge
df = pd.merge(
    df,
    df_defesa,    
    how='left',
    left_on='id_ctr',
    right_on='id_ctr'
)

# Resultados
df.head()

<br>

# Salva

In [ ]:
# Salva
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio_ctr.csv'),
    index=False,
)

# Resultados
df.head()